# Content-based recommendation

# Exercise 1
Based on the TF-IDF vectors obtained in the Exercise 2 from Session 4, represent each user in the same vector space. Amongst other feasible solutions, you can represent a user (user profile) by computing the weighted mean of the items vectors. Compute the cosine similarity for user 'A39WWMBA0299ZF' and all products in the training set not rated by the user. What are the top-5 recommended items for user 'A39WWMBA0299ZF'? Print out the top-5 items and their similarity score.  

In [2]:
from summary import Contentbased_recommendation, Evaluation
import gzip
import json
import pandas as pd

def getDF(path):
    def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield json.loads(l)
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df_1 = getDF('All_Beauty_5.json.gz')
df_2 = pd.read_json('meta_All_Beauty.json', lines=True)

In [3]:
preprocess = Contentbased_recommendation(ui_df=df_1, item_df=df_2)
predicetion_list = preprocess.predection()
evaluation = Evaluation(predicetion_list, df_1, df_2)

In [4]:
print(evaluation.rank_k(uid='A39WWMBA0299ZF',rank=5))

[('B019FWRG3C', array([0.4158461])), ('B00W259T7G', array([0.17710348])), ('B019809F9Y', array([0.10437818])), ('B00IJHY54S', array([0.08747406])), ('B0006O10P4', array([0.08561345]))]


# Exercise 2



Compute the systems’ hit rate based on the top-5, top-10 and top-20 recommendations, averaged over the total number of users. Remember that, as we are evaluating the system, you should compute the hit rate over the test set. How well/bad does this Content-based approach perform compared to the Collaborative Filtering?

In [5]:
kr_mean_5 = evaluation.mean_k(5, 'hr_k')
kr_mean_10 = evaluation.mean_k(10, 'hr_k')
kr_mean_20 = evaluation.mean_k(20, 'hr_k')

In [6]:
print(kr_mean_5)
print(kr_mean_10)
print(kr_mean_20)

0.4320337197049526
0.45732349841938885
0.5163329820864068


# Exercise 3

Repeat Exercise 1 and 2, this time representing the products and users in a word2vec vector space. You may use the gensim library and download the 300-dimension embeddings from Google. Source: https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

In [1]:
import gensim.downloader
word2vec_vectors = gensim.downloader.load('word2vec-google-news-300')



[--------------------------------------------------] 1.6% 26.3/1662.8MB downloaded

KeyboardInterrupt: 